In [7]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import math
import uuid

dataset = 'backdoor'
raw_file = "./data/{}.csv".format(dataset)

df = pd.read_csv(raw_file, header=None)
num_features = len(df.columns) - 1

print("Number of Features: {}".format(num_features))

Number of Features: 196


In [2]:
def partition_dataset(df, num_normal=20, num_anomalies=20):
    df_n = df[df.iloc[:,-1] == 1].sample(num_normal)
    df_a = df[df.iloc[:,-1] == -1].sample(num_anomalies)
    
    df.drop(df_n.index, inplace=True)
    df.drop(df_a.index, inplace=True)
    
    frames = [df_n, df_a]
    df_validation = pd.concat(frames)
    
    return df, df_validation

num_normal = 100
num_anomalies = 100

df_training, df_validation = partition_dataset(df, num_normal, num_anomalies)

# Save to partitions
key = str(uuid.uuid4())
print("Key: {}".format(key))
df_training.to_csv("partitions/{}-training-{}.csv".format(dataset, key), header=None)
df_validation.to_csv("partitions/{}-validation-{}.csv".format(dataset, key), header=None)

total = len(df_training)
num_normal = len(df_training[df_training.iloc[:,-1] == 1])
num_anomalies = len(df_training[df_training.iloc[:,-1] == -1])
contamination_ratio = (num_anomalies / total) * 100

# Get the vector of values disregarding labels
x_training = df_training.iloc[:,:-1].values
x_validation = df_validation.iloc[:,:-1].values

y_training = df_training.iloc[:,-1].values
y_validation = df_validation.iloc[:,-1].values

print("Total: {}".format(total))
print("Normal Count: {}".format(num_normal))
print("Anomaly Count: {}".format(num_anomalies))
print("Contamination Ratio: {}".format(contamination_ratio))

Key: eed45643-1cf0-4cd3-9ac9-b7da0538565e
Total: 48867
Normal Count: 45486
Anomaly Count: 3381
Contamination Ratio: 6.918779544477868


In [3]:
from pyod.models.ecod import ECOD

clf = ECOD()
clf.fit(x_training)

predictions = clf.predict(x_validation)
predictions = np.where(predictions == 1, -1, predictions)
predictions = np.where(predictions == 0, 1, predictions)

cm = confusion_matrix(y_validation, predictions)
tp = cm[0][0]
tn = cm[1][1]
fp = cm[0][1]
fn = cm[1][0]

mcc = ((tn * tp) - (fn * fp)) / math.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
f1 = tp / (tp + (0.5 * (fp + fn)))

print(classification_report(y_validation, predictions))
print("MCC: {}".format(mcc))
print("F1: {}".format(f1))

              precision    recall  f1-score   support

          -1       0.92      0.56      0.70       100
           1       0.68      0.95      0.79       100

    accuracy                           0.76       200
   macro avg       0.80      0.76      0.75       200
weighted avg       0.80      0.76      0.75       200

MCC: 0.5538572671642146
F1: 0.6956521739130435


In [4]:
from pyod.models.copod import COPOD

clf = COPOD()
clf.fit(x_training)

predictions = clf.predict(x_validation)
predictions = np.where(predictions == 1, -1, predictions)
predictions = np.where(predictions == 0, 1, predictions)

cm = confusion_matrix(y_validation, predictions)
tp = cm[0][0]
tn = cm[1][1]
fp = cm[0][1]
fn = cm[1][0]

mcc = ((tn * tp) - (fn * fp)) / math.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
f1 = tp / (tp + (0.5 * (fp + fn)))

print(classification_report(y_validation, predictions))
print("MCC: {}".format(mcc))
print("F1: {}".format(f1))

              precision    recall  f1-score   support

          -1       0.92      0.49      0.64       100
           1       0.65      0.96      0.78       100

    accuracy                           0.73       200
   macro avg       0.79      0.72      0.71       200
weighted avg       0.79      0.72      0.71       200

MCC: 0.5098188765330927
F1: 0.6405228758169934


In [5]:
import sys
import os

sys.path.append(os.path.join(os.path.abspath(''), '../pyneural/lib'))
sys.path.append(os.path.join(os.path.abspath(''), '../pyneural/modules'))

from autoencoder import Autoencoder
from train_autoencoder import TrainAutoencoder

batch_size = 5
learning_rate = 0.001
epochs = 50

params = {
    'layers':        [num_features, num_features - 3],
    'batch_size':    batch_size,
    'model_file':    '/home/ralampay/workspace/pyneural/models/ae-{}.pth'.format(dataset),
    'training_data': x_training,
    'epochs':        epochs,
    'learning_rate': learning_rate
}

cmd = TrainAutoencoder(params)
cmd.execute()

Training model...
Storing data to tensor...
Epoch: 0


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9774/9774 [00:14<00:00, 682.46it/s, loss=0.00182]


Ave Loss: 0.0023224307109249577
Epoch: 1


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9774/9774 [00:12<00:00, 765.95it/s, loss=0.000402]


Ave Loss: 0.000665433766621906
Epoch: 2


...

Ave Loss: 5.292586601699681e-05
Epoch: 48


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9774/9774 [00:16<00:00, 581.68it/s, loss=0.000121]


Ave Loss: 5.299243566180789e-05
Epoch: 49


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9774/9774 [00:14<00:00, 696.40it/s, loss=0.000122]


Ave Loss: 5.3065799207382316e-05


In [6]:
from auto_threshold_re import AutoThresholdRe
import torch

model = cmd.model
x_tensor = torch.tensor(x_training).float()
clf = AutoThresholdRe(x_tensor, model)
clf.execute()

print("Optimal Threshold: {}".format(clf.optimal_threshold))

predictions = clf.predict(torch.tensor(x_validation).float())

cm = confusion_matrix(y_validation, predictions)
tp = cm[0][0]
tn = cm[1][1]
fp = cm[0][1]
fn = cm[1][0]

mcc = ((tn * tp) - (fn * fp)) / math.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
f1 = tp / (tp + (0.5 * (fp + fn)))

print(classification_report(y_validation, predictions))
print("MCC: {}".format(mcc))
print("F1: {}".format(f1))

Optimal Threshold: 0.08697314356252367
              precision    recall  f1-score   support

          -1       0.55      0.97      0.71       100
           1       0.88      0.22      0.35       100

    accuracy                           0.59       200
   macro avg       0.72      0.59      0.53       200
weighted avg       0.72      0.59      0.53       200

MCC: 0.2872529994870127
F1: 0.7054545454545454
